In [1]:
from b2d.testbed.backend.jupyter import JupyterGui

In [2]:
import ipywidgets
import ipycanvas
import b2d
from plot_newtons_cradle import NewtonsCradle  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

Output()

In [ ]:
from plot_blender import Blender  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

In [ ]:
from plot_gauss_machine import GaussMachine  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

In [ ]:
from plot_color_mixing import ColorMixing  as Example
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)

In [ ]:
from plot_function_shape import FunctionShape  as Example
res = (400,400)
gui_settings = {
    "id": Example
}
Example.run(JupyterGui, gui_settings=gui_settings)